In [2]:
from pickle import dump
from random import choice

import networkx as nx
from pyvis.network import Network
from tqdm import tqdm

In [3]:
graphs_to_generate = 10000
p = .15

In [4]:
def create_graph(n):
    g = nx.erdos_renyi_graph(n, p)
    # dg = max(g.degree(n) for n in g)
    while True:
        for u in g:
            if g.degree[u] > 0:
                # print(f'd(u) = {g.degree[u]}')
                continue

            v = choice(list(g))
            if v == u:
                # print(f'{v} == {u}, D(g) = {dg}, n = {n}, V = {list(g)}')
                break

            g.add_edge(u, v)
            # print(f'+ e({u}, {v})')
            break
        else:
            break

    return g

In [5]:
def show(g, s):
    for i in g:
        g.nodes[i]['label'] = str(i)
        if i in s:
            g.nodes[i]['color'] = '#fb8500'

    # nt = Network('1000px', '850px', notebook=True)
    nt = Network(notebook=True)
    nt.from_nx(g)
    nt.toggle_physics(False)
    nt.options.edges.smooth.enabled = False
    nt.options.interaction.multiselect = True
    nt.options.interaction.selectConnectedEdges = False

    nt.show_buttons()
    return nt.show('nx.html')

In [6]:
import numpy as np
from scipy.optimize import milp, LinearConstraint


def milp_solve(g):
    # Solving MVC with MILP
    c = np.ones(len(g))
    A = np.zeros((len(g.edges), len(g)))
    for i, edge in enumerate(g.edges):
        v1, v2 = edge
        A[i, v1] = 1
        A[i, v2] = 1

    b_l = np.ones(len(g.edges))
    b_u = np.full_like(b_l, np.inf)

    constraints = LinearConstraint(A, b_l, b_u)
    integrality = np.ones_like(c)

    res = milp(c=c, constraints=constraints, integrality=integrality)
    mvc = {i for i, v in enumerate(res.x) if v}
    return mvc

In [7]:
for g_i in tqdm(range(graphs_to_generate), unit='graph'):
    g = create_graph(n=10)

   # conjunto indepente

    # Approximate Algorithm for Vertex Cover
    result = set()
    e_set = set(g.edges)
    n_list = sorted(g, key=lambda x: g.degree[x], reverse=True)
    while n_list:
        s = n_list.pop(0)
        result.add(s)

        # TODO use adj(s) instead of the whole e_set
        edges_cvrd = {e for e in e_set if e[0] == s or e[1] == s}
        nodes_cvrd = {u if u != s else v for u, v in edges_cvrd}
        e_set -= edges_cvrd
        for nc in nodes_cvrd:
            if nc in n_list:
                n_list.remove(nc)

    mvc = milp_solve(g)
    g.graph['mvc'] = [n in mvc for n in g]

    with open(f'./data/{g_i}', 'wb') as f:
        dump(g, f)

100%|██████████| 10000/10000 [00:07<00:00, 1386.51graph/s]


In [8]:
for g_i in tqdm(range(graphs_to_generate), unit='graph'):
    g = create_graph(n=10)

    # Approximate Algorithm for Vertex Cover
    # result = set()
    # e_set = set(g.edges)
    # n_list = sorted(g, key=lambda x: g.degree[x], reverse=True)
    # while n_list:
    #     s = n_list.pop(0)
    #     result.add(s)
    #
    #     # TODO use adj(s) instead of the whole e_set
    #     edges_cvrd = {e for e in e_set if e[0] == s or e[1] == s}
    #     nodes_cvrd = {u if u != s else v for u, v in edges_cvrd}
    #     e_set -= edges_cvrd
    #     for nc in nodes_cvrd:
    #         if nc in n_list:
    #             n_list.remove(nc)

    mvc = milp_solve(g)

    # check if valid vertex cover
    assert all(v1 in mvc or v2 in mvc for v1, v2 in g.edges)

    g.graph['mvc'] = [int(v in mvc) for v in g]

    with open(f'./data/{g_i}', 'wb') as f:
        dump(g, f)

100%|██████████| 10000/10000 [00:05<00:00, 1732.41graph/s]


In [9]:
show(g, g.graph['mvc'])

nx.html


In [10]:
show(g, milp_slt)

NameError: name 'milp_slt' is not defined